<center>
    <font size="5" color="steelblue"><b>Universidad Nacional de Colombia</b></font><br>
    <font size="5" color="steelblue"><b>Unidad de Informática - Facultad de Ciencias Económicas</b></font><br>
    <font size="5" color="steelblue"><b>Curso Libre Machine Learning con Python | 2023-1</b></font><br>
    <font size="5" color="steelblue"><b>Semana 4: Aprendizaje Supervisado Caso Aplicado</b></font><br>
    <font size="5" color="steelblue"><b>Estudiante Auxiliar: Jaime Andrés Fierro Aponte</b></font><br>
</center>

# Dependencias

In [ ]:
# Manejo de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
# from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# Machine Learning
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.over_sampling import SMOTE

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import (confusion_matrix, classification_report,
                             roc_curve, auc, balanced_accuracy_score)

# Conexión con Drive
from google.colab import drive

In [ ]:
# Establecer la conexión con mi drive
drive.mount('/content/drive')

In [ ]:
# Función para graficar el área bajo la curva ROC
def auc_roc(y_true, y_predicted):

  plt.figure(figsize=(5, 5))

  tpr,fpr,_ = roc_curve(y_true, y_predicted)
  roc_auc = 1 - auc(fpr, tpr)
  plt.plot(tpr, fpr, color='darkorange', lw=2,
          label='Curva ROC (area = %0.2f)' % roc_auc)
  plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
  plt.xlim([-0.05, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('% Falso Positivo')
  plt.ylabel('% Verdadero Positivo')
  plt.title('Curva ROC')
  plt.legend(loc='lower right')



# Importación de datos

El conjunto de datos recupera información sobre los clientes de un banco. Éstos datos fueron construídos para el análisis sobre la cancelación de productos creditios por parte de algunos clientes. Las variables disponibles son las siguientes:

* **CLIENTNUM**: Número de cliente. Identificador único del cliente titular de la cuenta
* **Attrition_Flag**: Actividad del cliente - Cliente que abandonó o cliente que aún existe
* **Customer_Age**: Variable demográfica - Edad del cliente en años
* **Gender**: Variable demográfica - M=Male, F=Female
* **Dependent_count**: Variable demográfica - Número de dependientes
* **Education_Level**: Variable demográfica**: Nivel educativo - Nivel educativo del titular de la cuenta (high school, college graduate, etc.)
* **Marital_Status**: Variable demográfica**: Estado civil - Married, Single, Divorced, Unknow
* **Income**: Variable demográfica**: Ingreso - Ingreso anual del titular de la cuenta
* **Card_Category**: Varible del producto - Tipo de la tarjeta (Blue, Silver, Gold, Platinum)
* **Months_on_book**: Periodo de relación con el banco
* **Total_Relationship_Count**: Número total de productos obtenidos por el cliente
* **Months_Inactive_12_mon**: Número de meses inactivo en los últimos 12 meses
* **Contacts_Count_12_mon**: Número de contactos en los últimos 12 meses
* **Credit_Limit**: Límite de crédito en la tarjeta de crédito
* **Total_Revolving_Bal**: Saldo rotatorio total en la tarjeta de crédito
* **Avg_Open_To_Buy**: Línea de crédito abierta para comprar (promedio de los últimos 12 meses)
* **Total_Amt_Chng_Q4_Q1**: Cambio en el monto de la transacción (Q4 sobre Q1)
* **Total_Trans_Amt**: Monto total de transacción (últimos 12 meses)
* **Total_Trans_Ct**: Cantidad total de transacciones (últimos 12 meses)
* **Total_Ct_Chng_Q4_Q1**: Cambio en la cantidad total de transacciones (Q4 sobre Q1)
* **Avg_Utilization_Ratio**: Ratio de uso promedio de la tarjeta

In [ ]:
# Importar los datos
bank_attrited = pd.read_excel('/content/drive/MyDrive/UIFCE/0. 2023-1 Curso Libre Machine Learning en Python/Datos/Bank Attrited Customers.xlsx')
bank_attrited

## Exploración Inicial

In [ ]:
# Tipo de objeto
type(bank_attrited)

In [ ]:
# Tipo de datos por variable
bank_attrited.dtypes

In [ ]:
# Dimensiones
bank_attrited.shape

In [ ]:
# Conteo de datos nulos
bank_attrited.isna().sum()
# Para tratar los datos nulos, revisar algoritmo MICE (Multiple Imputation in Chained Equations)

In [ ]:
# Información general
bank_attrited.info()

In [ ]:
# Eliminar variable identificadora
datos = bank_attrited.copy()
datos = datos.drop(columns='CLIENTNUM')\
             .dropna()

In [ ]:
datos.shape

# Selección de variables

## Análisis de correlación

In [ ]:
# Calcular la matriz de correlación
corr_matrix = datos.corr(numeric_only=True)

# Matriz de correlación reducida

# Variables que tienen alta correlación
high_corrs = [
              (i, j)
              for i in corr_matrix.columns
              for j in corr_matrix.index
              if (np.abs(corr_matrix.loc[i, j]) >= 0.7) & (i != j)
             ]

# Lista con las variables después del filtro
lista_high_corrs = []
for i in range(len(high_corrs)):
  el_1 = high_corrs[i][0]
  el_2 = high_corrs[i][1]
  lista_high_corrs.extend([el_1, el_2])

# Variables únicas de la lista de variables
vars_high_corrs = list(set(lista_high_corrs))

# Matríz de correlación reducida
small_corr_matrix = datos[vars_high_corrs].corr()

# Heatmap de correlación
plt.figure(figsize=(8, 5))
sns.heatmap(small_corr_matrix,
            annot=True,
            linewidths=0.5,
            linecolor='black',
            vmin=-1,
            vmax=1,
            fmt = '.3f',
            cmap='coolwarm')
plt.show()

Varias soluciones se pueden aplicar en los casos de correlaciones altas. Por ejemplo, probar con PCA en las variables que están muy correlacionadas, o análisis factorial. Se puede intentar transformar algunas de las variables con correlación, aplicando distintos tipos de funciones, como la logarítmica, curva power, curva S, entre otros, dependiendo de lo que tenga sentido analítico. Otra opción es sumar o promediar las variables, dependiendo de las características que tengan y del sentido analítico que se le de dentro del negocio.

In [ ]:
# Para el caso de ejemplo que estamos viendo, eliminaremos las variables correlacionadas
# con base en la experiencia
vars_no_corr = [
    'Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count',
    'Education_Level', 'Marital_Status', 'Income', 'Card_Category',
    'Total_Relationship_Count', 'Months_Inactive_12_mon',
    'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
    'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
    'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'
    ]

datos_no_corr = datos.copy()
datos_no_corr = datos_no_corr[vars_no_corr]

# Pre-procesamiento

## Estandarización de variables numéricas

In [ ]:
datos_no_corr['Total_Relationship_Count'].value_counts()

In [ ]:
datos_no_corr['Months_Inactive_12_mon'].value_counts()

In [ ]:
datos_no_corr['Contacts_Count_12_mon'].value_counts()

In [ ]:
num_vars = [
    'Customer_Age', 'Income', 'Credit_Limit', 'Total_Revolving_Bal',
    'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Ct_Chng_Q4_Q1',
    'Avg_Utilization_Ratio'
    ]
datos_num = datos_no_corr[num_vars]
datos_num.shape

In [ ]:
type(datos_num)

In [ ]:
datos_num

In [ ]:
# Crear instancia StandardScaler
scaler = StandardScaler()

# Estandarizar las variables
X_scaled = scaler.fit_transform(datos_num)

# Dimensiones de los datos estandarizados
X_scaled.shape

In [ ]:
type(X_scaled)

In [ ]:
X_scaled

## Codificación de variables categóricas

In [ ]:
categ_vars = [
    'Gender', 'Dependent_count', 'Education_Level',
    'Marital_Status', 'Card_Category', 'Total_Relationship_Count',
    'Months_Inactive_12_mon', 'Contacts_Count_12_mon'
    ]
datos_categ = datos_no_corr[categ_vars]
datos_categ.shape

In [ ]:
# Codificación
encoder = OneHotEncoder(sparse_output=False, drop='first')
X_encoded = encoder.fit_transform(datos_categ)

# Cantidad de características generadas
X_encoded.shape

In [ ]:
datos_no_corr['Gender'].value_counts()

In [ ]:
datos_no_corr['Card_Category'].value_counts()

In [ ]:
# Características categóricas codificadas
encoder.get_feature_names_out()

## Partición de los datos

In [ ]:
# Características a modelar
X = np.concatenate((X_scaled, X_encoded), axis=1)
X.shape

In [ ]:
# Conservar nombres de las características
feature_names = num_vars\
                + encoder.get_feature_names_out()\
                         .tolist()

In [ ]:
# Verificar cantidad de nombres con cantidad de características
len(feature_names)

In [ ]:
encoder_y = OneHotEncoder(sparse_output=False, drop='if_binary')
y = encoder_y.fit_transform(datos['Attrition_Flag'].values.reshape(-1, 1))

y

In [ ]:
datos['Attrition_Flag'] = datos['Attrition_Flag'].str.replace('Existing Customer', '1')
datos['Attrition_Flag'] = datos['Attrition_Flag'].str.replace('Attrited Customer', '0')
datos['Attrition_Flag'] = datos['Attrition_Flag'].astype(int)

y = datos['Attrition_Flag']

In [ ]:
# Partición de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.7,
                                                    random_state=42)

# Balanceo de clases

In [ ]:
# Clases desbalanceadas
plt.figure(figsize=(10, 5))
sns.countplot(x='Attrition_Flag', data=datos)
plt.xlabel('Attrition Flag')
plt.ylabel('Conteo')
plt.show()

In [ ]:
# Balanceo con sobremuestreo SMOTE

smote = SMOTE(k_neighbors=5, random_state=42)
X_train_balance, y_train_balance = smote.fit_resample(X_train, y_train)

class_counts = pd.Series(y_train_balance).value_counts()
plt.figure(figsize=(10, 5))
plt.bar(class_counts.index, class_counts, color=sns.color_palette('colorblind'))
plt.xticks(range(2))
plt.title('Distribución de las clases', size=22)
plt.show()

# Modelación

## Modelos lineales

### Naïve Bayes

In [ ]:
# Crear la instancia
nb_classifier = GaussianNB()
# Entrenar el modelo
nb_classifier.fit(X_train_balance, y_train_balance)

# Predicción del modelo sobre los datos de entrenamiento
y_train_pred_nb = nb_classifier.predict(X_train_balance)

# Reporte de métricas de evaluación para clasificación
print(classification_report(y_train_balance, y_train_pred_nb))

## Grid-Search K-Flod Cross Validation

In [ ]:
# KNN
grid_knn = {
    'n_neighbors':range(5, 11)
    }

gs_cv_knn = GridSearchCV(KNeighborsClassifier(),
                        param_grid=grid_knn,
                        verbose=1)

gs_cv_knn.fit(X_train_balance, y_train_balance)

print(f'Best Params: {gs_cv_knn.best_params_}')
print(f'Best Training Score: {gs_cv_knn.best_score_}')


In [ ]:
# Árbol de Decisión
grid_dt = {
    'max_depth':range(4, 21, 2),
    'criterion':['gini', 'entropy']
}

gs_cv_dt = GridSearchCV(DecisionTreeClassifier(),
                        param_grid=grid_dt,
                        verbose=1)

gs_cv_dt.fit(X_train_balance, y_train_balance)

print(f'Best Params: {gs_cv_dt.best_params_}')
print(f'Best Scoring: {gs_cv_dt.best_score_}')

# Evaluación

## Predicciones sobre datos de prueba

In [ ]:
# Predicciónes sobre las características de prueba

y_pred_nb = nb_classifier.predict(X_test)
y_pred_knn = gs_cv_knn.predict(X_test)
y_pred_dt = gs_cv_dt.predict(X_test)

## Reporte de clasificación

In [ ]:
# Métricas de desempeño
print(f'{"-"*60}\n')
print('Naïve Bayes')
print(classification_report(y_test, y_pred_nb))
print(f'{"-"*60}\n')
print('KNN')
print(classification_report(y_test, y_pred_knn))
print(f'{"-"*60}\n')
print('Decicision Tree')
print(classification_report(y_test, y_pred_dt))
print(f'{"-"*60}')

In [ ]:
# Accuracy balanceado

balanced_acc_dt = balanced_accuracy_score(y_test, y_pred_nb)
balanced_acc_rf = balanced_accuracy_score(y_test, y_pred_knn)
balanced_acc_xgb = balanced_accuracy_score(y_test, y_pred_dt)

print(f'Balanced Accuracy Score for Naïve Bayes Classifier: {balanced_acc_dt:.4f}')
print(f'Balanced Accuracy Score for KNN Classifier: {balanced_acc_rf:.4f}')
print(f'Balanced Accuracy Score for Decision Tree Classifier: {balanced_acc_xgb:.4f}')

# Ensambles

In [ ]:
# Bosque Aleatorio
grid_rf = {
    'criterion':['gini', 'entropy'],
    'max_depth':range(5, 21, 5),
    'n_estimators':range(40, 61, 10),
    }

gs_cv_rf = GridSearchCV(RandomForestClassifier(),
                        param_grid=grid_rf,
                        verbose=1,
                        return_train_score=True)

gs_cv_rf.fit(X_train_balance, y_train_balance)

gs_cv_rf.best_params_

In [ ]:
# XGBoost
grid_xgb = {
    'learning_rate': np.arange(0.01, 0.06, 0.02),
    'n_estimators': range(40, 61, 10),
    'max_depth': range(5, 8)
}


gs_cv_xgb = GridSearchCV(XGBClassifier(),
                         param_grid=grid_xgb,
                         verbose=1,
                         return_train_score=True)

gs_cv_xgb.fit(X_train, y_train)

gs_cv_xgb.best_params_


In [ ]:
# Predicciónes sobre las características de prueba

y_pred_rf = gs_cv_rf.predict(X_test)
y_pred_xgb = gs_cv_xgb.predict(X_test)

In [ ]:
# Métricas de desempeño
print(f'{"-"*60}\n')
print('Random Forest')
print(classification_report(y_test, y_pred_rf))
print(f'{"-"*60}\n')
print('XGBoost')
print(classification_report(y_test, y_pred_xgb))


In [ ]:
# Accuracy balanceado

balanced_acc_dt = balanced_accuracy_score(y_test, y_pred_rf)
balanced_acc_rf = balanced_accuracy_score(y_test, y_pred_xgb)

print(f'Balanced Accuracy Score for Random Forest Classifier: {balanced_acc_dt:.4f}')
print(f'Balanced Accuracy Score for XGBoost Classifier: {balanced_acc_rf:.4f}')
